In [1]:
# !sudo apt install ffmpeg
# !python3 -m pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=23)

INFO: Pandarallel will run on 23 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
MFAOutputBasePath = '/home/devanshb_it_17/Downloads/montreal-forced-aligner/outeval'
import pandas as pd
metadf = pd.read_csv('metaData.csv')
from os import path
def txtGridFromTxt(txtPath):
    pathTokens = txtPath.split('/')
    txtGridPath = MFAOutputBasePath + '/' + pathTokens[-2] + '/' + pathTokens[-2] + pathTokens[-1][:-4] + ".TextGrid"
    if path.exists(txtGridPath):
        return txtGridPath
    return "delete"
metadf["txtGridPath"] = metadf['Path'].apply(txtGridFromTxt)
metadf = metadf[metadf["txtGridPath"] != "delete"]
metadf.shape

(31961, 4)

In [3]:
# MetaDFinList = []
# for index,item in metadf.iterrows():
#     MetaDFinList.append((item['Path'], item['Content']))
metadf.loc[0]["txtGridPath"]

'/home/devanshb_it_17/Downloads/montreal-forced-aligner/outeval/s6rJLXq1Re0/s6rJLXq1Re050002.TextGrid'

In [4]:
dfContainingIllion = metadf[metadf.Content.str.contains('illion',case=False)]
import textgrids
###################### Actually corrupt the frames #################################
def CorruptionWordTSForViseme(vidFiles):
    tg = textgrids.TextGrid(vidFiles['txtGridPath'])
    thisWordTG = {}
    overlaps = []
    for syll in tg['words']:
        # Convert Praat to Unicode in the label
        label = syll.text.transcode()
        # Print label and syllable duration, CSV-like
        if label.find("illion") == -1:
            continue   
        return str(syll.xmin) + " " + str(syll.xmax)
    return "0 0"

def CorruptionTSForViseme(vidFiles):
    WordTS = vidFiles["wordTS"].split()
    diff = float(WordTS[1]) - float(WordTS[0])
    CorruptStart = float(WordTS[0]) + 0.2 * diff
    CorruptEnd = float(WordTS[1]) - 0.2 * diff
    return f"{CorruptStart:.2f} {CorruptEnd:.2f}"

dfContainingIllion["wordTS"] = dfContainingIllion.parallel_apply(CorruptionWordTSForViseme, axis=1)
dfContainingIllion["corruptTS"] = dfContainingIllion.parallel_apply(CorruptionTSForViseme, axis=1)
dfContainingIllion.head()

/home/devanshb_it_17/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/devanshb_it_17/.local/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Path,Content,txtGridPath,wordTS,corruptTS
307,307,/home/devanshb_it_17/Downloads/trainval/rugSZI...,7 MILLION OF THESE HAVE MADE THEIR WAY TO A 12...,/home/devanshb_it_17/Downloads/montreal-forced...,0.56 0.85,0.62 0.79
536,536,/home/devanshb_it_17/Downloads/trainval/WpldYJ...,SO HERE'S A PROBLEM FOUR MILLION BABIES AROUND...,/home/devanshb_it_17/Downloads/montreal-forced...,2.13 2.47,2.20 2.40
550,550,/home/devanshb_it_17/Downloads/trainval/H1fvr9...,WHY ARE WE SPENDING 80 BILLION DOLLARS ON A PR...,/home/devanshb_it_17/Downloads/montreal-forced...,1.5 2.02,1.60 1.92
756,756,/home/devanshb_it_17/Downloads/trainval/Mr8nvX...,BECAUSE WHEN THE GOVERNMENT SAYS THAT A FAMILY...,/home/devanshb_it_17/Downloads/montreal-forced...,3.19 3.48,3.25 3.42
805,805,/home/devanshb_it_17/Downloads/trainval/B905La...,BUT SECOND WE HAVE ACCIDENTALLY MADE IT SO THA...,/home/devanshb_it_17/Downloads/montreal-forced...,2.81 3.25,2.90 3.16


In [5]:
# allow word level corruptions of all strings having a word of at least this length
WORDLENGTH_THRESHOLD = 10

def getWordOfMinLetters(string, threshold):
    words = string.split()
    maxlenword = max(words, key=len)
    if (len(maxlenword) > threshold):
        return maxlenword
    else:
        return "not_valid"

def CorruptionWordTSForWLC(vidFiles):
    tg = textgrids.TextGrid(vidFiles['txtGridPath'])
    thisWordTG = {}
    overlaps = []
    for syll in tg['words']:
        # Convert Praat to Unicode in the label
        label = syll.text.transcode()
        # Print label and syllable duration, CSV-like
        if label.upper() != (vidFiles["corruptions"]):
            continue   
        return (str(syll.xmin) + " " + str(syll.xmax))
    return "0.0 0.0"

def CorruptionTSForWLC(vidFiles):
    WordTS = str(vidFiles["wordTS"]).split()
    diff = float(WordTS[1]) - float(WordTS[0])
    CorruptStart = float(WordTS[0]) + 0.2 * diff
    CorruptEnd = float(WordTS[1]) - 0.2 * diff
    return f"{CorruptStart:.2f} {CorruptEnd:.2f}"

isFitForWordLevelCorruptions = metadf['Content'].apply(lambda s: getWordOfMinLetters(s, WORDLENGTH_THRESHOLD))
dfFitForWordLevelCorruptions = metadf.copy()
dfFitForWordLevelCorruptions["corruptions"] = isFitForWordLevelCorruptions
dfFitForWordLevelCorruptions = dfFitForWordLevelCorruptions[isFitForWordLevelCorruptions != "not_valid"]
dfFitForWordLevelCorruptions["wordTS"] = dfFitForWordLevelCorruptions.parallel_apply(CorruptionWordTSForWLC, axis=1)
dfFitForWordLevelCorruptions["corruptTS"] = dfFitForWordLevelCorruptions.parallel_apply(CorruptionTSForWLC, axis=1)
dfFitForWordLevelCorruptions.size

36260

In [6]:
# allow word level corruptions of all strings having two consecutive words of at least this length
WORDLENGTH_THRESHOLD = 6
import re
def getConsecWordOfMinLetters(string, threshold):
    pattern = '(\w{' + str(threshold) + ',}\s\w{' + str(threshold) + ',})'
    match = re.search(pattern, string, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return "not_valid"

def CorruptionWordTSForMWLC(vidFiles):
    tg = textgrids.TextGrid(vidFiles['txtGridPath'])
    thisWordTG = {}
    overlaps = []
    intervals = []
    prevMatch = False
    prevTS = 0
    for syll in tg['words']:
        # Convert Praat to Unicode in the label
        label = syll.text.transcode()
        # Print label and syllable duration, CSV-like
        if (prevMatch == True) and (label.upper() == vidFiles["corruptions"].split()[1]):
            return (str(prevTS) + " " + str(syll.xmax))
        if prevMatch == True:
            prevMatch = False
        if label.upper() == (vidFiles["corruptions"].split()[0]):
            prevMatch = True  
            prevTS = syll.xmin
    return "0 0"

    return "0.0 0.0"

def CorruptionTSForMWLC(vidFiles):
    WordTS = str(vidFiles["wordsTS"]).split()
    diff = float(WordTS[1]) - float(WordTS[0])
    CorruptStart = float(WordTS[0]) + 0.2 * diff
    CorruptEnd = float(WordTS[1]) - 0.2 * diff
    return f"{CorruptStart:.2f} {CorruptEnd:.2f}"
    
isFitForMultiWordLevelCorruptions = metadf['Content'].apply(lambda s: getConsecWordOfMinLetters(s, WORDLENGTH_THRESHOLD))
dfFitForMultiWordLevelCorruptions = metadf.copy()
dfFitForMultiWordLevelCorruptions["corruptions"] = isFitForMultiWordLevelCorruptions
dfFitForMultiWordLevelCorruptions = dfFitForMultiWordLevelCorruptions[isFitForMultiWordLevelCorruptions != "not_valid"]
dfFitForMultiWordLevelCorruptions["wordsTS"] = dfFitForMultiWordLevelCorruptions.parallel_apply(CorruptionWordTSForMWLC, axis=1)
dfFitForMultiWordLevelCorruptions["corruptTS"] = dfFitForMultiWordLevelCorruptions.parallel_apply(CorruptionTSForMWLC, axis=1)
dfFitForMultiWordLevelCorruptions.head()
print(dfFitForMultiWordLevelCorruptions.shape)

(10421, 7)


In [7]:
dfFitForMultiWordLevelCorruptions.head()

,Unnamed: 0,Path,Content,txtGridPath,corruptions,wordsTS,corruptTS
2,2,/home/devanshb_it_17/Downloads/trainval/xtRIvS...,I HAD 5000 FOLLOWERS ON POLYVORE AND 9000 COMB...,/home/devanshb_it_17/Downloads/montreal-forced...,COMBINED FOLLOWERS,3.52 4.37,3.69 4.20
4,4,/home/devanshb_it_17/Downloads/trainval/xtRIvS...,I WAS AND STILL AM TERRIFIED OF PUBLIC SPEAKING,/home/devanshb_it_17/Downloads/montreal-forced...,PUBLIC SPEAKING,2.5 3.31,2.66 3.15
10,10,/home/devanshb_it_17/Downloads/trainval/ynSrWm...,SO I NEEDED TO START CONSULTING IN MYSELF AND ...,/home/devanshb_it_17/Downloads/montreal-forced...,TRUSTING MYSELF,2.68 3.6,2.86 3.42
11,11,/home/devanshb_it_17/Downloads/trainval/ynSrWm...,BUT AT THAT MOMENT I ACKNOWLEDGED DIFFERENT EM...,/home/devanshb_it_17/Downloads/montreal-forced...,ACKNOWLEDGED DIFFERENT,1.53 2.5,1.72 2.31
12,12,/home/devanshb_it_17/Downloads/trainval/o8NPll...,THIS COULD ACTUALLY IMPROVE THIS,/home/devanshb_it_17/Downloads/montreal-forced...,ACTUALLY IMPROVE,0.62 1.36,0.77 1.21


In [11]:
# !mkdir corruptionVids
# !mkdir corruptionVids/eval
dfContainingIllion.to_csv("corruptionVids/eval/corruptionsForIllion.csv", index=False)
dfFitForWordLevelCorruptions.to_csv("corruptionVids/eval/corruptionsForWordLevel.csv", index=False)
dfFitForMultiWordLevelCorruptions.to_csv("corruptionVids/eval/corruptionsForMultiWordLevel.csv", index=False)

In [14]:
dfFitForMultiWordLevelCorruptions

,Unnamed: 0,Path,Content,txtGridPath,corruptions,wordsTS,corruptTS
2,2,/home/devanshb_it_17/Downloads/trainval/xtRIvS...,I HAD 5000 FOLLOWERS ON POLYVORE AND 9000 COMB...,/home/devanshb_it_17/Downloads/montreal-forced...,COMBINED FOLLOWERS,3.52 4.37,3.69 4.20
4,4,/home/devanshb_it_17/Downloads/trainval/xtRIvS...,I WAS AND STILL AM TERRIFIED OF PUBLIC SPEAKING,/home/devanshb_it_17/Downloads/montreal-forced...,PUBLIC SPEAKING,2.5 3.31,2.66 3.15
10,10,/home/devanshb_it_17/Downloads/trainval/ynSrWm...,SO I NEEDED TO START CONSULTING IN MYSELF AND ...,/home/devanshb_it_17/Downloads/montreal-forced...,TRUSTING MYSELF,2.68 3.6,2.86 3.42
11,11,/home/devanshb_it_17/Downloads/trainval/ynSrWm...,BUT AT THAT MOMENT I ACKNOWLEDGED DIFFERENT EM...,/home/devanshb_it_17/Downloads/montreal-forced...,ACKNOWLEDGED DIFFERENT,1.53 2.5,1.72 2.31
12,12,/home/devanshb_it_17/Downloads/trainval/o8NPll...,THIS COULD ACTUALLY IMPROVE THIS,/home/devanshb_it_17/Downloads/montreal-forced...,ACTUALLY IMPROVE,0.62 1.36,0.77 1.21
...,...,...,...,...,...,...,...
31967,31967,/home/devanshb_it_17/Downloads/trainval/ISjk9n...,AND I THINK IT SUPREMELY ILLUSTRATES THE PREMI...,/home/devanshb_it_17/Downloads/montreal-forced...,SUPREMELY ILLUSTRATES,0 0,0.00 0.00
31968,31968,/home/devanshb_it_17/Downloads/trainval/ISjk9n...,WHEN THEY DO DO INTENTIONAL PHYSICAL ACTIVITY ...,/home/devanshb_it_17/Downloads/montreal-forced...,INTENTIONAL PHYSICAL,0.98 1.86,1.16 1.68
31969,31969,/home/devanshb_it_17/Downloads/trainval/QPC5SP...,IT REMOVES CARBON DIOXIDE FROM THE AIR,/home/devanshb_it_17/Downloads/montreal-forced...,REMOVES CARBON,0.41 1.32,0.59 1.14
31975,31975,/home/devanshb_it_17/Downloads/trainval/M1yCvK...,WE ENDED UP WITH THE RESIDENTIAL SCHOOLS,/home/devanshb_it_17/Downloads/montreal-forced...,RESIDENTIAL SCHOOLS,0.74 1.73,0.94 1.53


In [18]:
dfFitForMultiWordLevelCorruptions[dfFitForMultiWordLevelCorruptions["Path"].str.contains("0d6i")]

,Unnamed: 0,Path,Content,txtGridPath,corruptions,wordsTS,corruptTS
12437,12437,/home/devanshb_it_17/Downloads/trainval/0d6iSv...,TWO THINGS HAPPENED AS WE COPED WITH LOSS AND ...,/home/devanshb_it_17/Downloads/montreal-forced...,THINGS HAPPENED,0.38 1.37,0.58 1.17
12439,12439,/home/devanshb_it_17/Downloads/trainval/0d6iSv...,AND IT'S THESE THREE THINGS IMMIGRANTS' VOTES ...,/home/devanshb_it_17/Downloads/montreal-forced...,THINGS IMMIGRANTS,0 0,0.00 0.00
12441,12441,/home/devanshb_it_17/Downloads/trainval/0d6iSv...,SO IN 2000 WHEN IT WAS ANNOUNCED THAT THE CITI...,/home/devanshb_it_17/Downloads/montreal-forced...,CITIZENSHIP APPLICATION,3.26 4.67,3.54 4.39
